# Student Loan Risk with Deep Learning

In [39]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [40]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [41]:
# Review the data types associated with the columns
loans_df.dtypes

payment_history           float64
location_parameter        float64
stem_degree_score         float64
gpa_ranking               float64
alumni_success            float64
study_major_code          float64
time_to_completion        float64
finance_workshop_score    float64
cohort_ranking            float64
total_loan_score          float64
financial_aid_score       float64
credit_ranking              int64
dtype: object

In [42]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

credit_ranking
1    855
0    744
Name: count, dtype: int64

### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [43]:
# Define the target set y using the credit_ranking column
y = loans_df['credit_ranking'].to_numpy()

# Display a sample of y
y[:5]

array([0, 0, 0, 1, 0])

In [44]:
# Define features set X by selecting all columns but credit_ranking
X = loans_df.drop(columns=['credit_ranking'])

# Review the features DataFrame
X.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4


### Step 3: Split the features and target sets into training and testing datasets.


In [45]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [46]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
scaler.fit(X_train)

# Fit the scaler to the features training dataset
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [47]:
# Define the the number of inputs (features) to the model
number_of_features = X_train_scaled.shape[1]

# Review the number of features
number_of_features

11

In [48]:
# Define the number of hidden nodes for the first hidden layer
first_hidden_layer_nodes = 6

# Define the number of hidden nodes for the second hidden layer
second_hidden_layer_nodes = 3

# Define the number of neurons in the output layer
output_layer_nodes = 1

In [49]:
# Create the Sequential model instance
model = Sequential()

# Add the first hidden layer
model.add(Dense(units=first_hidden_layer_nodes, activation='relu', input_shape=(number_of_features,)))

# Add the second hidden layer
model.add(Dense(units=second_hidden_layer_nodes, activation='relu'))

# Add the output layer to the model specifying the number of output neurons and activation function
model.add(Dense(units=output_layer_nodes, activation='sigmoid'))

In [50]:
# Display the Sequential model summary
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 6)                 72        
                                                                 
 dense_7 (Dense)             (None, 3)                 21        
                                                                 
 dense_8 (Dense)             (None, 1)                 4         
                                                                 
Total params: 97 (388.00 Byte)
Trainable params: 97 (388.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [51]:
# Compile the Sequential model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [52]:
# Fit the model using 50 epochs and the training data
model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_data=(X_test_scaled, y_test))

Epoch 1/50
38/38 [==============================] - 1s 9ms/step - loss: 0.6900 - accuracy: 0.5488 - val_loss: 0.6738 - val_accuracy: 0.5825
Epoch 2/50
38/38 [==============================] - 0s 4ms/step - loss: 0.6664 - accuracy: 0.5830 - val_loss: 0.6517 - val_accuracy: 0.6200
Epoch 3/50
38/38 [==============================] - 0s 3ms/step - loss: 0.6454 - accuracy: 0.6105 - val_loss: 0.6325 - val_accuracy: 0.6475
Epoch 4/50
38/38 [==============================] - 0s 3ms/step - loss: 0.6269 - accuracy: 0.6330 - val_loss: 0.6153 - val_accuracy: 0.6700
Epoch 5/50
38/38 [==============================] - 0s 3ms/step - loss: 0.6107 - accuracy: 0.6597 - val_loss: 0.6004 - val_accuracy: 0.6825
Epoch 6/50
38/38 [==============================] - 0s 3ms/step - loss: 0.5958 - accuracy: 0.6672 - val_loss: 0.5871 - val_accuracy: 0.6900
Epoch 7/50
38/38 [==============================] - 0s 4ms/step - loss: 0.5827 - accuracy: 0.6739 - val_loss: 0.5757 - val_accuracy: 0.6900
Epoch 8/50
38/38 [==

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [53]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
loss, accuracy = model.evaluate(X_test_scaled, y_test)

# Display the model loss and accuracy results
print(f"Loss: {loss}, Accuracy: {accuracy}")

13/13 [==============================] - 0s 3ms/step - loss: 0.5216 - accuracy: 0.7350
Loss: 0.5216113924980164, Accuracy: 0.7350000143051147


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [54]:
# Set the model's file path
model_path = "student_loans.keras"

# Export your model to a keras file
model.save(model_path)

---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [55]:
# Set the model's file path


# Load the model to a new object
loaded_model = tf.keras.models.load_model(model_path)

### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [56]:
# Make predictions with the test data
predictions = loaded_model.predict(X_test_scaled)

# Display a sample of the predictions
predictions[:5]

13/13 [==============================] - 0s 1ms/step


array([[0.52015966],
       [0.28410125],
       [0.8752582 ],
       [0.730672  ],
       [0.9597989 ]], dtype=float32)

In [57]:
# Save the predictions to a DataFrame and round the predictions to binary results
predictions_df = pd.DataFrame(predictions, columns=["predictions"])
predictions_df["predictions"] = predictions_df["predictions"].round()

predictions_df

,predictions
0,1.0
1,0.0
2,1.0
3,1.0
4,1.0
...,...
395,1.0
396,0.0
397,1.0
398,0.0


### Step 4: Display a classification report with the y test data and predictions

In [58]:
# Print the classification report with the y test data and predictions
print(classification_report(y_test, predictions_df["predictions"]))

              precision    recall  f1-score   support

           0       0.69      0.78      0.73       188
           1       0.78      0.69      0.73       212

    accuracy                           0.73       400
   macro avg       0.74      0.74      0.73       400
weighted avg       0.74      0.73      0.73       400



---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.

2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.

3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.

**1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.**

**Answer:**
To build an effective recommendation system for student loan options, the following data would be relevant and appropriate:

Student Demographics:

Age: Helps to understand the stage of education.
Gender: For demographic profiling and tailored recommendations.
Location: Different regions might have different loan options and terms.
Educational Background:

Current Institution: Some loans are specific to certain schools.
Degree Program: Different programs might have different funding requirements.
GPA: Higher GPAs might qualify for better loan terms.
Year of Study: Freshman, sophomore, etc., can influence the loan amount needed.
Financial Information:

Income: Student's and/or parent's income to assess need.
Credit Score: To determine eligibility and interest rates.
Existing Loans: To understand current debt burden.
Loan Preferences:

Loan Amount Needed: The total amount the student needs.
Repayment Term Preference: Short-term vs. long-term loans.
Interest Rate Preference: Fixed vs. variable rates.
Deferment Options: Preferences for deferment during school or after graduation.
Historical Data:

Past Loan Repayment History: If available, to assess reliability.
Employment Prospects: Likelihood of employment post-graduation, which can affect repayment ability.


**2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.**

**Answer:**
Given the data chosen for the recommendation system, content-based filtering would be the most suitable approach. Here's why:

Content-Based Filtering: This method recommends loans based on the attributes of the student and their preferences. Since we have detailed information about the student's financial background, educational status, and loan preferences, we can match these attributes directly with the features of available loan options.
Relevance: The recommendation is directly based on individual student's profile and loan attributes.
Independence: Does not rely on other users' data, which might be sparse or unavailable in some cases.


**3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.**

**Answer:**
Data Privacy and Security:

Concern: Handling sensitive financial and personal information about students requires strict compliance with data protection laws (e.g., GDPR, FERPA).
Solution: Implement robust encryption methods, access controls, and ensure compliance with relevant regulations to protect student data.
Data Quality and Completeness:

Concern: Incomplete or inaccurate data can lead to poor recommendations. For example, missing income information or outdated credit scores can skew the system's output.
Solution: Implement data validation checks, regular updates, and encourage users to provide complete and accurate information. Use imputation techniques for handling missing data and ensure continuous data quality monitoring.

**Conclusion:**

Building a student loan recommendation system requires comprehensive and sensitive data to tailor loan options effectively. Using content-based filtering leverages the detailed attributes of each student to provide personalized loan recommendations. However, challenges such as ensuring data privacy and maintaining high data quality are critical to address for the system to be reliable and secure.
